In [1]:
import os
import urllib
import zipfile
import pandas as pd
from pandas.api import types as pdt
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, cross_val_score, TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import xgboost as xgb 
import joblib
import traceback

AIR_QUALITY_URL = "https://archive.ics.uci.edu/ml/machine-learning-databases/00360/AirQualityUCI.zip"
AIR_QUALITY_DATA_DIR = os.path.join('data', 'air_quality_data')

In [2]:
def download_file(url, dest_path):
    try:
        urllib.request.urlretrieve(url, dest_path)
        print(f'Data downloaded successfully to {dest_path}')
        return True
    except Exception as e:
        print(f'Failed to download data: {e}')
        return False

In [3]:
def extract_zip(zip_path, extract_dir):
    try:
        with zipfile.ZipFile(zip_path) as z:
            z.extractall(extract_dir)
            print(f"Data extracted to {extract_dir}")
        return True
    except Exception as e:
        print(f'Failed to extract zip: {e}')
        return False

In [4]:
def load_csv(csv_path):
    try:
        df = pd.read_csv(csv_path, sep=';', decimal=',')
        print(f'Data loaded successfully from {csv_path}')
        return df
    except Exception as e:
        print(f"Failed to read CSV: {e}")
        return pd.DataFrame()

In [5]:
def fetch_air_quality_data(air_quality_url=AIR_QUALITY_URL, aq_data_dir=AIR_QUALITY_DATA_DIR):
    os.makedirs(aq_data_dir, exist_ok=True)
    csv_path = os.path.join(aq_data_dir, 'AirQualityUCI.csv')
    zip_path = os.path.join(aq_data_dir, 'AirQualityUCI.zip')
    
    if not os.path.exists(csv_path):
        if not os.path.exists(zip_path):
            if not download_file(air_quality_url, zip_path):
                return pd.DataFrame()
        if not extract_zip(zip_path, aq_data_dir):
            return pd.DataFrame()
    return load_csv(csv_path)

In [6]:
def get_air_quality_clean_data(df=None, target='C6H6(GT)'):
    """Cleans the air quality dataset by handling missing values and removing unnecessary columns"""
    if df is None:
        df = fetch_air_quality_data()
    if df.empty:
        raise ValueError('Data not loaded - the file is empty')
    
    # Strip column names and remove entirely empty columns
    df.columns = df.columns.str.strip()
    df.dropna(axis=1, how='all', inplace=True)
    
    if target not in df.columns:
        available_columns = list(df.columns)
        raise ValueError(f'Target column "{target}" not found. Available columns: {available_columns}')
    
    df.replace(-200, np.nan, inplace=True)
    sensor_columns = df.columns.difference(['Date', 'Time'])
    df.dropna(subset=sensor_columns, how='all', inplace=True)
    df.dropna(subset=[target], inplace=True)
    
    if 'NMHC(GT)' in df.columns:
        df.drop('NMHC(GT)', axis=1, inplace=True)
        
    # Check if we have enough data after cleaning
    if len(df) == 0:
        raise ValueError('No data remaining after cleaning')

    return df

In [7]:
def add_datetime_column(df, date_col='Date', time_col='Time'):
    """Combines separate 'Date' and 'Time' columns into a single datetime column"""
    df = df.copy()
    
    if date_col not in df.columns or time_col not in df.columns:
        print(f"Error: Missing columns. Please check if '{date_col}' and '{time_col}' exist")
        return df
    
    df['DateTime'] = pd.to_datetime(
        df[date_col] + ' ' + df[time_col], 
        format='%d/%m/%Y %H.%M.%S',
        errors='coerce'
    )
    
    if df['DateTime'].isna().any():
        print(f"Warning: {df['DateTime'].isna().sum()} invalid DateTime rows dropped")
        df.dropna(subset=['DateTime'], inplace=True)
    
    df.drop(columns=[date_col, time_col], inplace=True)
    return df

In [ ]:
def seasonal_train_test_split(df, datetime_col='DateTime', test_months=3):
    df = df.copy()
    df['__year'] = df[datetime_col].dt.year
    df['__month'] = df[datetime_col].dt.month
    year_month_df = df[['__year', '__month']]
    
    unique_periods = year_month_df.drop_duplicates().sort_values(['__year', '__month'])
    test_periods = unique_periods.tail(test_months)
    
    test_mask = year_month_df.apply(tuple, axis=1).isin(test_periods.apply(tuple, axis=1))
    
    train_idx=df.loc[~test_mask].index.tolist()
    test_idx = df.loc[test_mask].index.tolist()
    
    return train_idx, test_idx

In [ ]:
class DateTimeTransformer(BaseEstimator, TransformerMixin):
    """Custom transformer to extract and encode cyclical temporal features from 'Date' and 'Time' columns"""
    def __init__(self, datetime_col='DateTime', use_day_of_year=True, use_year=False, use_day=True):
        self.datetime_col = datetime_col
        self.use_day_of_year = use_day_of_year
        self.use_year = use_year
        self.use_day = use_day

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Check if required column exist
        if self.datetime_col not in X.columns:
            raise ValueError("Could not parse datetime from Date and Time columns")
        
        dt_series = X[self.datetime_col]
        
        # Handle any parsing errors
        if dt_series.isna().all():
            raise ValueError("Could not parse datetime from Date and Time columns")
        
        features = {
            'Month_sin': np.sin(2 * np.pi * dt_series.dt.month / 12), 
            'Month_cos': np.cos(2 * np.pi * dt_series.dt.month / 12),
            'DayOfWeek_sin': np.sin(2 * np.pi * dt_series.dt.dayofweek / 7),
            'DayOfWeek_cos': np.cos(2 * np.pi * dt_series.dt.dayofweek / 7),
            'Hour_sin': np.sin(2 * np.pi * dt_series.dt.hour / 24),
            'Hour_cos': np.cos(2 * np.pi * dt_series.dt.hour / 24),
        }

        if self.use_day_of_year:
            features['DayOfYear_sin'] = np.sin(2 * np.pi * dt_series.dt.dayofyear / 365)
            features['DayOfYear_cos'] = np.cos(2 * np.pi * dt_series.dt.dayofyear / 365)      
        if self.use_year:
            features['Year'] = dt_series.dt.year
        if self.use_day:
            features['Day_sin'] = np.sin(2 * np.pi * dt_series.dt.day / 31)
            features['Day_cos'] = np.cos(2 * np.pi * dt_series.dt.day / 31)

        return pd.DataFrame(features, index=X.index)

In [10]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

dt_pipeline = Pipeline([
    ('datetime', DateTimeTransformer()),
    ('imputer', SimpleImputer(strategy='mean'))
])

In [11]:
models = {
    'forest_reg': {
        'estimator': RandomForestRegressor(random_state=42),
        'param_grid': [
            {
                'n_estimators': [200, 300, 400],
                'max_features': ['log2', 'sqrt', None, 0.5, 0.6, 0.7],
                'max_depth': [None, 10, 20],
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4]
            }
        ]
    },
    'xgboost': {
        'estimator': xgb.XGBRegressor(random_state=42),
        'param_grid': [
            {
                'n_estimators': [200, 300, 400],
                'max_depth': [4, 6, 8],
                'learning_rate': [0.05, 0.1, 0.3],
                'subsample': [0.7, 0.9, 1],
                'colsample_bytree': [0.6, 0.8, 1],
                'gamma': [0, 0.1, 0.3],
                'reg_lambda': [1, 5, 10],
                'reg_alpha': [0, 0.1, 0.5, 1],
                'min_child_weight': [1, 5, 10, 20]
            }
        ]
    },
    'hist_grad_boost': {
        'estimator': HistGradientBoostingRegressor(random_state=42),
        'param_grid': [
            {
                'max_iter': [200, 300, 400],
                'max_depth': [4, 6, 8],
                'learning_rate': [0.05, 0.1, 0.3],
                'l2_regularization': [1, 5, 10],
                'min_samples_leaf': [1, 5, 10, 20]
            }
        ]
    }
}

In [12]:
def train_models(models, X_train, y_train, full_pipeline, cv=TimeSeriesSplit(n_splits=10, gap=24), skipped_models=None):
    """Trains the specified models using RandomizedSearchCV and returns the trained models"""
    trained_models = {}
    skipped_models = skipped_models or []

    for model_name, model_info in models.items():
        if model_name in skipped_models:
            continue
        
        print(f'Training model: {model_name.upper()}')

        estimator_pipeline = Pipeline([
            ('preprocessor', full_pipeline),
            ('regressor', model_info['estimator'])
        ])
        
        param_grid = {}
        for param, values in model_info['param_grid'][0].items():
            param_grid[f'regressor__{param}'] = values
            
        random_search = RandomizedSearchCV(
            estimator_pipeline,
            param_distributions=param_grid,
            cv=cv,
            scoring={
                'rmse': 'neg_root_mean_squared_error',
                'r2': 'r2'
            },
            refit='rmse',
            random_state=42,
            return_train_score=True,
            n_jobs=1,
            n_iter=300,
            verbose=2
        )
        try:
            random_search.fit(X_train, y_train)
            trained_models[model_name] = {
                'search_cv': random_search,
                'best_estimator': random_search.best_estimator_
            }
            print(f'{model_name.upper()}: training completed')
        except Exception as e:
            print(f'Error during training {model_name}: {e}')
            continue
            
    return trained_models

In [13]:
def save_models(trained_models, save_dir='models', leaderboard_flag=True):
    """Saves the trained models and optionally generates a leaderboard"""
    os.makedirs(save_dir, exist_ok=True)
    leaderboard = []

    for model_name, model_info in trained_models.items():
        model_path = os.path.join(save_dir, model_name)
        os.makedirs(model_path, exist_ok=True)

        try:
            joblib.dump(model_info['search_cv'], os.path.join(model_path, 'search_cv.joblib'))
            joblib.dump(model_info['best_estimator'], os.path.join(model_path, 'best_model.joblib'))
            print(f'{model_name}: saved')

            if leaderboard_flag:
                search_cv = model_info['search_cv']
                best_idx = search_cv.best_index_
                
                leaderboard.append({
                    'Model': model_name,
                    'CV_RMSE': round(search_cv.cv_results_['mean_test_rmse'][best_idx], 4),
                    'CV_R2': round(search_cv.cv_results_['mean_test_r2'][best_idx], 4),
                    'Best_Params': str(search_cv.best_params_)
                })
        except Exception as e:
            print(f'Error during saving {model_name}: {e}')

    if leaderboard_flag and leaderboard:
        leaderboard_df = pd.DataFrame(leaderboard).sort_values('CV_RMSE').reset_index(drop=True)
        leaderboard_path = os.path.join(save_dir, 'cv_leaderboard.csv')
        leaderboard_df.to_csv(leaderboard_path, index=False)
        print(f'\nLeaderboard saved to {leaderboard_path}')
    elif not leaderboard_flag:
        print('\nLeaderboard generation skipped')

In [14]:
def load_models(models, save_dir):
    """Loads trained models from the specified directory"""
    loaded_models = {}
    for model_name in models.keys():
        search_cv_path = os.path.join(save_dir, model_name, 'search_cv.joblib')
        best_model_path = os.path.join(save_dir, model_name, 'best_model.joblib')

        if os.path.exists(search_cv_path) and os.path.exists(best_model_path):
            try:
                loaded_models[model_name] = {
                    'search_cv': joblib.load(search_cv_path),
                    'best_estimator': joblib.load(best_model_path)
                }
                print(f'{model_name}: loaded from disk')
            except Exception as e:
                print(f'Error loading model {model_name}: {e}')
    return loaded_models

In [15]:
def train_save_models(models, X_train, y_train, full_pipeline, save_dir='models', leaderboard_flag=True, cv=None, force_retrain=False):
    """Trains and saves the specified models, optionally generating a leaderboard"""
    if cv is None:
        cv = TimeSeriesSplit(n_splits=10, gap=24)

    loaded_models = load_models(models, save_dir)
    skipped_model_names = list(loaded_models.keys()) if not force_retrain else []

    if force_retrain:
        print('\nForce retrain is ON — all models will be retrained.\n')
        loaded_models = {}
        skipped_model_names = []
    elif skipped_model_names:
        print('\nAlready trained models found and will be skipped:')
        for model in skipped_model_names:
            print(f'  - {model}')

    trained_models = train_models(
        models, X_train, y_train, full_pipeline,
        cv=cv, skipped_models=skipped_model_names
    )
            
    all_models = {**loaded_models, **trained_models}
    save_models(all_models, save_dir=save_dir, leaderboard_flag=leaderboard_flag)
    return all_models

In [16]:
if __name__ == '__main__':
    try:
        # Load and clean data
        air_quality = get_air_quality_clean_data()
        air_quality = add_datetime_column(air_quality)
        air_quality = air_quality.sort_values('DateTime').reset_index(drop=True)
        
        target = 'C6H6(GT)'
        X = air_quality.drop(target, axis=1)
        y = air_quality[target]
        
        # Split indices
        train_idx, test_idx = seasonal_train_test_split(
            air_quality, datetime_col='DateTime',  test_months=3
        )
        
        # Prepare train and test sets: combine X and y, sort by DateTime, and reset indices 
        train = pd.concat([X.loc[train_idx].copy(), y.loc[train_idx].copy()], axis=1)
        train = train.sort_values('DateTime').reset_index(drop=True)
        X_train = train.drop(target, axis=1)
        y_train = train[target]
        
        test = pd.concat([X.loc[test_idx].copy(), y.loc[test_idx].copy()], axis=1)
        test = test.sort_values('DateTime').reset_index(drop=True)
        X_test = test.drop(target, axis=1)
        y_test = test[target]
        
        print("Train max date:", X_train['DateTime'].max())
        print("Test min date:", X_test['DateTime'].min())
        assert X_train['DateTime'].max() < X_test['DateTime'].min()
        
        air_quality_num_columns = X_train.select_dtypes(include=[np.number]).columns.tolist()
        air_quality_dt_columns = ['DateTime']
        
        # Create preprocessing pipeline
        full_pipeline = ColumnTransformer([
            ('num', num_pipeline, air_quality_num_columns),
            ('dt', dt_pipeline, air_quality_dt_columns) 
        ])

        trained_models = train_save_models(
            models,
            X_train,
            y_train,
            full_pipeline,
            save_dir='models',
            leaderboard_flag=True,
            cv=TimeSeriesSplit(n_splits=10, gap=24),
            force_retrain=True
        )
        
        test_results = []
        for model_name, model_info in trained_models.items():
            try:
                best_model = model_info['best_estimator']
                y_pred = best_model.predict(X_test)
                rmse = root_mean_squared_error(y_test, y_pred)
                r2 = r2_score(y_test, y_pred)
                test_results.append({
                    'Model': model_name,
                    'RMSE': round(rmse, 4), 
                    'R2Score': round(r2, 4)
                })
            except Exception as e:
                print(f"Error evaluating {model_name}: {e}")

        test_results_df = pd.DataFrame(test_results).sort_values('RMSE').reset_index(drop=True)
        test_results_df.to_csv('models/test_results.csv', index=False) # Save test results
        test_results_df
    except Exception as e:
        print(f'An error occurred: {e}')
        traceback.print_exc()

Data loaded successfully from data\air_quality_data\AirQualityUCI.csv
Train max date: 2005-01-31 23:00:00
Test min date: 2005-02-01 00:00:00


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\User\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator RandomizedSearchCV 

forest_reg: loaded from disk
xgboost: loaded from disk


C:\Users\User\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:442: InconsistentVersionWarning: Trying to unpickle estimator HistGradientBoostingRegressor from version 1.5.1 when using version 1.7.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


Error loading model hist_grad_boost: Can't get attribute '__pyx_unpickle_CyHalfSquaredError' on <module 'sklearn._loss._loss' from 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python312\\site-packages\\sklearn\\_loss\\_loss.cp312-win_amd64.pyd'>

Force retrain is ON — all models will be retrained.

Training model: FOREST_REG
Fitting 10 folds for each of 300 candidates, totalling 3000 fits
[CV] END regressor__max_depth=20, regressor__max_features=0.5, regressor__min_samples_leaf=2, regressor__min_samples_split=10, regressor__n_estimators=200; total time=   1.0s
[CV] END regressor__max_depth=20, regressor__max_features=0.5, regressor__min_samples_leaf=2, regressor__min_samples_split=10, regressor__n_estimators=200; total time=   1.3s
[CV] END regressor__max_depth=20, regressor__max_features=0.5, regressor__min_samples_leaf=2, regressor__min_samples_split=10, regressor__n_estimators=200; total time=   2.3s
[CV] END regressor__max_depth=20, regressor__max_features=0.5, regressor__min_sample

KeyboardInterrupt: 